## Part 1: Generating Training and Testing Data
Before we can get started with differential privacy models, we first need to get the data to train and test the models.  
To do that, we will take the grade distributions from the UW-Madison Registrar's office and randomly sample them.

In [95]:
DATA_FOLDER = './data'

In [96]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
import numpy as np
import math

In [97]:
sns.set()

### Reading the Dataset

In [98]:
def plot():
    plt.figure(dpi=200, figsize=(8, 5))

def read_file_into_pd(file_name):
    return pd.read_csv(os.path.join(DATA_FOLDER, file_name))

In [99]:
grade_distributions = read_file_into_pd('grade_distributions.csv')
course_offerings = read_file_into_pd('course_offerings.csv')
courses = read_file_into_pd('courses.csv')
cs_courses = pd.read_csv('cs.csv', names=['name'])

In [100]:
grade_distributions.head()

,course_offering_uuid,section_number,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,s_count,u_count,cr_count,n_count,p_count,i_count,nw_count,nr_count,other_count
0,344b3ebe-da7e-314c-83ed-9425269695fd,1,105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,1,158,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,ea3b717c-d66b-30dc-8b37-964d9688295f,1,139,12,2,0,3,0,0,0,0,0,0,0,0,0,0,0
3,075da420-5f49-3dd0-93df-13e3c152e1b1,1,87,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,2b4e216d-a728-3713-8c7c-19afffc6b2fd,1,70,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [101]:
course_offerings.head()

,uuid,course_uuid,term_code,name
0,344b3ebe-da7e-314c-83ed-9425269695fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1092,Cooperative Education Prog
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1082,Cooperative Education Prog
2,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog
3,075da420-5f49-3dd0-93df-13e3c152e1b1,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1114,Cooperative Education Prog
4,2b4e216d-a728-3713-8c7c-19afffc6b2fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1104,Cooperative Education Prog


In [102]:
courses.head()

,uuid,name,number
0,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,Cooperative Education Program,1
1,c070a84f-648f-351d-9499-5d0e30ad02cc,Cooperative Education/Co-op in Life Sciences C...,1
2,e6b4b7ae-0e0b-3aa5-9d77-7fcd90c9cfa3,Cooperative Education Program,1
3,8f63bde1-ff7f-3fe7-9901-862908bf134c,Workshop in Dance Activity,1
4,f3541888-584a-3923-9ce7-6341ff3d84a1,Cooperative Education/Co-op in Agricultural & ...,1


In [103]:
cs_courses.head()

,name
0,PROGRAMMING I
1,INTRODUCTION TO COMPUTATION
2,DATA PROGRAMMING I
3,INTRODUCTION TO DISCRETE MATHEMATICS
4,DIGITAL SOCIETY: THE IMPACT OF COMPUTERS AND C...


### Calculate the weight range to bias student distributions

In [104]:
num_grades = len(distribution_renaming_dictionary)
weight_limit = 0.5
weight_range = np.linspace(start=weight_limit, stop=-weight_limit, num=num_grades)

In [113]:
distribution_renaming_dictionary = {
    "a_count": "A",
    "ab_count": "AB",
    "b_count": "B",
    "bc_count": "BC",
    "c_count": "C",
    "d_count": "D",
    "f_count": "F",
}

distribution_grades_to_drop = ["s_count","u_count","cr_count","n_count","p_count","i_count","nw_count","nr_count","other_count"]

def get_ids_for_course_name(name):
    return courses[courses['name'].str.lower() == name.lower()].uuid.tolist()

def get_offering_ids_for_course_ids(ids):
    return course_offerings[course_offerings.course_uuid.isin(ids)].uuid.tolist()

def get_ids_for_course_offering_name(name):
    return course_offerings[course_offerings.name.str.lower() == name.lower()].uuid.tolist()

def get_grades_for_course_offerings(course_offering_ids):
    return grade_distributions[grade_distributions.course_offering_uuid.isin(course_offering_ids)] \
        .sum().drop(['course_offering_uuid', 'section_number']).drop(distribution_grades_to_drop) \
        .rename(distribution_renaming_dictionary)

def get_grade_distribution_for_course_name(name):
    course_ids = get_ids_for_course_name(name)
    offering_ids = get_offering_ids_for_course_ids(course_ids)
    return get_grades_for_course_offerings(offering_ids)

def get_grade_distribution_for_course_offering_name(name):
    offering_ids = get_ids_for_course_offering_name(name)
    return get_grades_for_course_offerings(offering_ids)

def plot_grade_distribution(distribution, course_name):
    plot()
    plt.title(course_name)
    sns.barplot(x=distribution.index, y=distribution.values)
    
def sample_distribution(distribution):
    cumulative_sum = distribution.cumsum()
    r = random.randint(0, cumulative_sum['F'])
    for (index, value) in cumulative_sum.items():
        if value >= r:
            return index

def sample_courses(courses):
    cumulative_sum = courses['students'].cumsum()
    highest_value = cumulative_sum.iloc[len(filtered_cs_courses)-1]
    r = random.randint(0, highest_value)
    for (index, value) in cumulative_sum.items():
        if value >= r:
            return index

def sample_distribution_biased(distribution, bias):
    distribution_copy = distribution.copy()
    for ((index, value), weight) in zip(distribution.items(), weight_range):
        delta = value * bias * weight
        new_value = value + delta
        distribution_copy[index] = math.floor(new_value)
    cumulative_sum = distribution_copy.cumsum()
    r = random.randint(1, cumulative_sum['F']+1)
    for (index, value) in cumulative_sum.items():
        if value >= r:
            return index

### Drop all courses with no students

In [114]:
filtered_cs_courses = pd.DataFrame(columns=['name', 'students'])
for course in cs_courses['name']:
    num_students = get_grade_distribution_for_course_name(course).sum()
    if (num_students != 0):
        filtered_cs_courses = filtered_cs_courses.append({'name': course, 'students': num_students}, ignore_index=True)

In [115]:
print("Dropped %d courses after filtering." % (len(cs_courses) - len(filtered_cs_courses)))
print("Now using %d courses." % len(filtered_cs_courses))

Dropped 29 courses after filtering.
Now using 94 courses.


In [116]:
filtered_cs_courses = filtered_cs_courses.sort_values(by='students', ascending=False)
filtered_cs_courses.head()

,name,students
52,DIRECTED STUDY,7231
4,INTRODUCTION TO COMPUTER ENGINEERING,5185
2,INTRODUCTION TO DISCRETE MATHEMATICS,3842
10,MACHINE ORGANIZATION AND PROGRAMMING,3660
9,DIGITAL SYSTEM FUNDAMENTALS,3222


In [121]:
def generate_transcripts(num_students=100_000):
    transcripts = pd.DataFrame(columns=filtered_cs_courses['name'])
    for i in range(num_students):
        random_courses = set([])
        student_transcript = pd.Series(index=transcripts.columns)
        while len(random_courses) < 8:
            random_course_number = sample_courses(filtered_cs_courses)
            name = filtered_cs_courses.iloc[random_course_number]['name']
            distribution = get_grade_distribution_for_course_name(name)
            random_courses.add(name)
            bias = random.uniform(-2, 2)
            student_transcript[name] = sample_distribution_biased(distribution, bias)
        transcripts = transcripts.append(student_transcript, ignore_index=True)
        if i%100 == 0:
            print("%d/%d..." % (i, num_students))
    print("Done!")
    transcripts.to_csv('transcripts.csv')

In [122]:
generate_transcripts(num_students=50_000)

0/50000...
10/50000...
20/50000...
30/50000...
40/50000...
50/50000...
60/50000...
70/50000...
80/50000...
90/50000...
100/50000...
110/50000...
120/50000...
130/50000...
140/50000...
150/50000...
160/50000...
170/50000...
180/50000...
190/50000...
200/50000...
210/50000...
220/50000...
230/50000...
240/50000...
250/50000...
260/50000...
270/50000...
280/50000...
290/50000...
300/50000...
310/50000...
320/50000...
330/50000...
340/50000...
350/50000...
360/50000...
370/50000...
380/50000...
390/50000...
400/50000...
410/50000...
420/50000...
430/50000...
440/50000...
450/50000...
460/50000...
470/50000...
480/50000...
490/50000...
500/50000...
510/50000...
520/50000...
530/50000...
540/50000...
550/50000...
560/50000...
570/50000...
580/50000...
590/50000...
600/50000...
610/50000...
620/50000...
630/50000...
640/50000...
650/50000...
660/50000...
670/50000...
680/50000...
690/50000...
700/50000...
710/50000...
720/50000...
730/50000...
740/50000...
750/50000...
760/50000...
770/50000.

5940/50000...
5950/50000...
5960/50000...
5970/50000...
5980/50000...
5990/50000...
6000/50000...
6010/50000...
6020/50000...
6030/50000...
6040/50000...
6050/50000...
6060/50000...
6070/50000...
6080/50000...
6090/50000...
6100/50000...
6110/50000...
6120/50000...
6130/50000...
6140/50000...
6150/50000...
6160/50000...
6170/50000...
6180/50000...
6190/50000...
6200/50000...
6210/50000...
6220/50000...
6230/50000...
6240/50000...
6250/50000...
6260/50000...
6270/50000...
6280/50000...
6290/50000...
6300/50000...
6310/50000...
6320/50000...
6330/50000...
6340/50000...
6350/50000...
6360/50000...
6370/50000...
6380/50000...
6390/50000...
6400/50000...
6410/50000...
6420/50000...
6430/50000...
6440/50000...
6450/50000...
6460/50000...
6470/50000...
6480/50000...
6490/50000...
6500/50000...
6510/50000...
6520/50000...
6530/50000...
6540/50000...
6550/50000...
6560/50000...
6570/50000...
6580/50000...
6590/50000...
6600/50000...
6610/50000...
6620/50000...
6630/50000...
6640/50000...
6650/5

11680/50000...
11690/50000...
11700/50000...
11710/50000...
11720/50000...
11730/50000...
11740/50000...
11750/50000...
11760/50000...
11770/50000...
11780/50000...
11790/50000...
11800/50000...
11810/50000...
11820/50000...
11830/50000...
11840/50000...
11850/50000...
11860/50000...
11870/50000...
11880/50000...
11890/50000...
11900/50000...
11910/50000...
11920/50000...
11930/50000...
11940/50000...
11950/50000...
11960/50000...
11970/50000...
11980/50000...
11990/50000...
12000/50000...
12010/50000...
12020/50000...
12030/50000...
12040/50000...
12050/50000...
12060/50000...
12070/50000...
12080/50000...
12090/50000...
12100/50000...
12110/50000...
12120/50000...
12130/50000...
12140/50000...
12150/50000...
12160/50000...
12170/50000...
12180/50000...
12190/50000...
12200/50000...
12210/50000...
12220/50000...
12230/50000...
12240/50000...
12250/50000...
12260/50000...
12270/50000...
12280/50000...
12290/50000...
12300/50000...
12310/50000...
12320/50000...
12330/50000...
12340/5000

17150/50000...
17160/50000...
17170/50000...
17180/50000...
17190/50000...
17200/50000...
17210/50000...
17220/50000...
17230/50000...
17240/50000...
17250/50000...
17260/50000...
17270/50000...
17280/50000...
17290/50000...
17300/50000...
17310/50000...
17320/50000...
17330/50000...
17340/50000...
17350/50000...
17360/50000...
17370/50000...
17380/50000...
17390/50000...
17400/50000...
17410/50000...
17420/50000...
17430/50000...
17440/50000...
17450/50000...
17460/50000...
17470/50000...
17480/50000...
17490/50000...
17500/50000...
17510/50000...
17520/50000...
17530/50000...
17540/50000...
17550/50000...
17560/50000...
17570/50000...
17580/50000...
17590/50000...
17600/50000...
17610/50000...
17620/50000...
17630/50000...
17640/50000...
17650/50000...
17660/50000...
17670/50000...
17680/50000...
17690/50000...
17700/50000...
17710/50000...
17720/50000...
17730/50000...
17740/50000...
17750/50000...
17760/50000...
17770/50000...
17780/50000...
17790/50000...
17800/50000...
17810/5000

22620/50000...
22630/50000...
22640/50000...
22650/50000...
22660/50000...
22670/50000...
22680/50000...
22690/50000...
22700/50000...
22710/50000...
22720/50000...
22730/50000...
22740/50000...
22750/50000...
22760/50000...
22770/50000...
22780/50000...
22790/50000...
22800/50000...
22810/50000...
22820/50000...
22830/50000...
22840/50000...
22850/50000...
22860/50000...
22870/50000...
22880/50000...
22890/50000...
22900/50000...
22910/50000...
22920/50000...
22930/50000...
22940/50000...
22950/50000...
22960/50000...
22970/50000...
22980/50000...
22990/50000...
23000/50000...
23010/50000...
23020/50000...
23030/50000...
23040/50000...
23050/50000...
23060/50000...
23070/50000...
23080/50000...
23090/50000...
23100/50000...
23110/50000...
23120/50000...
23130/50000...
23140/50000...
23150/50000...
23160/50000...
23170/50000...
23180/50000...
23190/50000...
23200/50000...
23210/50000...
23220/50000...
23230/50000...
23240/50000...
23250/50000...
23260/50000...
23270/50000...
23280/5000

28090/50000...
28100/50000...
28110/50000...
28120/50000...
28130/50000...
28140/50000...
28150/50000...
28160/50000...
28170/50000...
28180/50000...
28190/50000...
28200/50000...
28210/50000...
28220/50000...
28230/50000...
28240/50000...
28250/50000...
28260/50000...
28270/50000...
28280/50000...
28290/50000...
28300/50000...
28310/50000...
28320/50000...
28330/50000...
28340/50000...
28350/50000...
28360/50000...
28370/50000...
28380/50000...
28390/50000...
28400/50000...
28410/50000...
28420/50000...
28430/50000...
28440/50000...
28450/50000...
28460/50000...
28470/50000...
28480/50000...
28490/50000...
28500/50000...
28510/50000...
28520/50000...
28530/50000...
28540/50000...
28550/50000...
28560/50000...
28570/50000...
28580/50000...
28590/50000...
28600/50000...
28610/50000...
28620/50000...
28630/50000...
28640/50000...
28650/50000...
28660/50000...
28670/50000...
28680/50000...
28690/50000...
28700/50000...
28710/50000...
28720/50000...
28730/50000...
28740/50000...
28750/5000

33560/50000...
33570/50000...
33580/50000...
33590/50000...
33600/50000...
33610/50000...
33620/50000...
33630/50000...
33640/50000...
33650/50000...
33660/50000...
33670/50000...
33680/50000...
33690/50000...
33700/50000...
33710/50000...
33720/50000...
33730/50000...
33740/50000...
33750/50000...
33760/50000...
33770/50000...
33780/50000...
33790/50000...
33800/50000...
33810/50000...
33820/50000...
33830/50000...
33840/50000...
33850/50000...
33860/50000...
33870/50000...
33880/50000...
33890/50000...
33900/50000...
33910/50000...
33920/50000...
33930/50000...
33940/50000...
33950/50000...
33960/50000...
33970/50000...
33980/50000...
33990/50000...
34000/50000...
34010/50000...
34020/50000...
34030/50000...
34040/50000...
34050/50000...
34060/50000...
34070/50000...
34080/50000...
34090/50000...
34100/50000...
34110/50000...
34120/50000...
34130/50000...
34140/50000...
34150/50000...
34160/50000...
34170/50000...
34180/50000...
34190/50000...
34200/50000...
34210/50000...
34220/5000

39030/50000...
39040/50000...
39050/50000...
39060/50000...
39070/50000...
39080/50000...
39090/50000...
39100/50000...
39110/50000...
39120/50000...
39130/50000...
39140/50000...
39150/50000...
39160/50000...
39170/50000...
39180/50000...
39190/50000...
39200/50000...
39210/50000...
39220/50000...
39230/50000...
39240/50000...
39250/50000...
39260/50000...
39270/50000...
39280/50000...
39290/50000...
39300/50000...
39310/50000...
39320/50000...
39330/50000...
39340/50000...
39350/50000...
39360/50000...
39370/50000...
39380/50000...
39390/50000...
39400/50000...
39410/50000...
39420/50000...
39430/50000...
39440/50000...
39450/50000...
39460/50000...
39470/50000...
39480/50000...
39490/50000...
39500/50000...
39510/50000...
39520/50000...
39530/50000...
39540/50000...
39550/50000...
39560/50000...
39570/50000...
39580/50000...
39590/50000...
39600/50000...
39610/50000...
39620/50000...
39630/50000...
39640/50000...
39650/50000...
39660/50000...
39670/50000...
39680/50000...
39690/5000

44500/50000...
44510/50000...
44520/50000...
44530/50000...
44540/50000...
44550/50000...
44560/50000...
44570/50000...
44580/50000...
44590/50000...
44600/50000...
44610/50000...
44620/50000...
44630/50000...
44640/50000...
44650/50000...
44660/50000...
44670/50000...
44680/50000...
44690/50000...
44700/50000...
44710/50000...
44720/50000...
44730/50000...
44740/50000...
44750/50000...
44760/50000...
44770/50000...
44780/50000...
44790/50000...
44800/50000...
44810/50000...
44820/50000...
44830/50000...
44840/50000...
44850/50000...
44860/50000...
44870/50000...
44880/50000...
44890/50000...
44900/50000...
44910/50000...
44920/50000...
44930/50000...
44940/50000...
44950/50000...
44960/50000...
44970/50000...
44980/50000...
44990/50000...
45000/50000...
45010/50000...
45020/50000...
45030/50000...
45040/50000...
45050/50000...
45060/50000...
45070/50000...
45080/50000...
45090/50000...
45100/50000...
45110/50000...
45120/50000...
45130/50000...
45140/50000...
45150/50000...
45160/5000

49970/50000...
49980/50000...
49990/50000...
Done!
